<a href="https://colab.research.google.com/github/hassanSattariNia/prediction_time_series/blob/main/Albert_V2_PEFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch peft scikit-learn

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from peft import get_peft_model, LoraConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"]
)

In [ ]:
# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
model = get_peft_model(model, lora_config)
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # Directory to save checkpoints and logs
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
)

trainer.train()


# Evaluate manually to check if eval_accuracy is calculated
results = trainer.evaluate()
print("Evaluation Results:", results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.722200,No log
2,0.695800,No log
3,0.668900,No log


Evaluation Results: {'eval_runtime': 0.7901, 'eval_samples_per_second': 25.314, 'eval_steps_per_second': 2.531, 'epoch': 3.0}


In [ ]:

# Load dataset
dataset = load_dataset("glue", "mrpc")
print(f"Train set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")

Train set size: 3668
Validation set size: 408
Test set size: 1725


In [ ]:
# Preprocess function
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [ ]:
# Evaluate on validation set
results = trainer.evaluate()
print("Validation Results:", results)

Validation Results: {'eval_runtime': 0.7897, 'eval_samples_per_second': 25.325, 'eval_steps_per_second': 2.532, 'epoch': 3.0}


In [ ]:
# Evaluate on test set
test_results = trainer.evaluate(eval_dataset=tokenized_dataset['test'])
print("Test Results:", test_results)

# Evaluate on training set
train_results = trainer.evaluate(eval_dataset=tokenized_dataset['train'])
print("Train Results:", train_results)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from peft import get_peft_model, LoraConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    # Print the metrics for debugging purposes
    print(f"Metrics: Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "key", "value"]
)

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=2)
model = get_peft_model(model, lora_config)
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

# Load dataset
dataset = load_dataset("glue", "mrpc")
print(f"Train set size: {len(dataset['train'])}")
print(f"Validation set size: {len(dataset['validation'])}")
print(f"Test set size: {len(dataset['test'])}")

# Preprocess function
def preprocess_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # Changed to 'f1' to match the compute_metrics output
    greater_is_better=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'].shuffle(seed=42).select(range(200)),
    eval_dataset=tokenized_dataset['validation'].shuffle(seed=42).select(range(20)),
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate on validation set
results = trainer.evaluate()
print("Validation Results:", results)

# Evaluate on test set
test_results = trainer.evaluate(eval_dataset=tokenized_dataset['test'])
print("Test Results:", test_results)

# Evaluate on training set
train_results = trainer.evaluate(eval_dataset=tokenized_dataset['train'])
print("Train Results:", train_results)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Train set size: 3668
Validation set size: 408
Test set size: 1725


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.689400,No log


KeyError: "The `metric_for_best_model` training argument is set to 'eval_f1', which is not found in the evaluation metrics. The available evaluation metrics are: []. Consider changing the `metric_for_best_model` via the TrainingArguments."